## GMRES

### Overview

#### What is GMRES?

- GMRES is an iterative solver for a system of linear equations: $Ax=b$.
- GMRES generalizes the Conjugate gradient method to *asymmetric* matrix $A$.
- The name is short for *generalized minimum residual* method.
- Why do we discuss that in a chapter of least square?
  - Part of its algorithm involves solving least square problem.

### Why do we care?

- GMRES can deal with asymmetric matrix, which the conjugate gradient method fails with.
- GMRES is a good choice for the solution of large, sparse, asymmetric (square) linear system $Ax=b$. [Sauer (2017) p. 235]
- GMRES deals with ill-conditioning using orthogonality. [Sauer (2017) p. 235]

### The method

#### Idea

- $K_j:=\{r, Ar, A^2 r, \cdots, A^j r\}$ is called *Krylov* space. 
- The approximate solution $x_k$ at $k$-th iteration is the best approximation of the true solution $x$ in $K_{k}$. 
  - Conjugate gradient method uses the similar idea. And they both belong to *Krylov methods*.
- As $k$ increases, $K_k$ expands and the approximation gets better and better.
  - In theory, GMRES is a direct method: It terminates at $n$-th iteration with the exact solution if $A$ is nonsingular. [Sauer (2017) p. 237]

**Algorithm** (GMRES; Sauer (2017) p. 235)

- Given
  - $A$: $n$-by-$n$ matrix
  - $b$: vector of length $n$
- Initialize
  - $x_0$: initial guess
  - $r=b-A x_0$: initial residual 
  - $q_1=r /\|r\|_2$
- Compute
  - **for** $k=1,2, \ldots, m$
    - $y=A q_k$
    - **for** $j=1,2, \ldots, k$
      - $h_{j k}=q_j^T y $
      - $y=y-h_{j k} q$
    - $h_{k+1, k}=\|y\|_2$ (If $h_{k+1, k}=0$, skip next line and terminate at bottom.)
    - $q_{k+1}=y / h_{k+1, k}$
    - Minimize $\left\|H_k c_k- [\|r\|_2, 0, 0, \ldots, 0]^T \right\|_2$ for $c_k$
    - $x_k=Q_k c_k+x_0$

At $k$-th step, 
- $[\|r\|_2, 0, 0, \ldots, 0]$ is length $k+1$.
- $c_k$ is of length $k$.
- $H_k$ is of size $(k+1)\times k$ and given by 
 
$$
H = \left[\begin{array}{cccc}
h_{11} & h_{12} & \cdots & h_{1 k} \\
h_{21} & h_{22} & \cdots & h_{2 k} \\
& h_{32} & \cdots & h_{3 k} \\
& \ddots & \vdots \\
& & & h_{k+1, k}
\end{array}\right]
$$

  - $Q_k$ is of size $n\times k$ and given by

$$
\left[\begin{array}{c:c:c} 
& & \\
& & \\
q_1 & \cdots & q_k \\
& & \\
& &
\end{array}\right]
$$


**Outline**

- At step $k$ of the method, we enlarge the Krylov space by adding $A^k r$, 
- reorthogonalize the basis (i.e., inner loop for modified Gram-Schimidt), 
- and then use least squares to find the best approximation in $K_k$.
  - This is done by finding $x_{add}$ ($Q_k c_k$ in the algorith) and add it to $x_0$.
  - This step involves its own details.

**Details 1**


- It holds that $AQ_k = Q_{k+1} H_k$ for each $k$
  - This is a consequence of Gram-Schmidt (inner loop): with $y=A q_j$, we have orthogonal decomposition $y = \underbrace{(q_1^T y)}_{h_{1,j}} q_1 + \underbrace{(q_2^T y)}_{h_{2,j}} q_2 + \cdots + \underbrace{(q_{j+1}^T y)}_{h_{j+1,j}}q_{j+1}$ for $j=1,2,\cdots,k$.
    - This decomposition is used in the third equality below.
    - As a result, $q_{j+1} \in \mathrm{span} \{q_i\}_{i=1}^{j} \cup \{Aq_j\}$
    - We are assuming that there is no technical difficulty such as $y=A q_{j+1}\in\mathrm{span}\{q_i\}_{i=1}^{j}$.

$$
\begin{split}
AQ_k &= A\left[\begin{array}{c:c:c} 
  & & \\
  & & \\
  q_1 & \cdots & q_k \\
  & & \\
  & &
  \end{array}\right] \\
&=\left[\begin{array}{c:c:c} 
  & & \\
  & & \\
  Aq_1 & \cdots & Aq_k \\
  & & \\
  & &
  \end{array}\right]
\\
&=
\left[
	\begin{array}{c:c:c:c}
		Q_{k+1} 
		\begin{pmatrix}
  			h_{11}\\
  			h_{21}\\
  			\\
  			\\
  		\end{pmatrix} 
  		&
  		Q_{k+1} 
		\begin{pmatrix}
  			h_{12}\\
  			h_{22}\\
  			h_{32}\\
  			\\
  		\end{pmatrix}
	  	&
 		\cdots 
		&
  		Q_{k+1} 
		\begin{pmatrix}
  			h_{12}\\
  			h_{22}\\
  			h_{32}\\
  			\vdots \\
  			h_{j+1, j}
  		\end{pmatrix}
	\end{array}
  \right]
\\
& = Q_{k+1}
\left[\begin{array}{c:c:c:c}
h_{11} & h_{12} & \cdots & h_{1 k} \\
h_{21} & h_{22} & \cdots & h_{2 k} \\
& h_{32} & \cdots & h_{3 k} \\
& \ddots & \vdots \\
& & & h_{k+1, k}
\end{array}\right]
\\
&= Q_{k+1}H_k
% \left[\begin{array}{l:l:l:l} 
% & & & \\
% q_1 & \cdots & q_k & q_{k+1} \\
% & & & \\
% & & &
% \end{array}\right]\left[\begin{array}{cccc}
% h_{11} & h_{12} & \cdots & h_{1 k} \\
% h_{21} & h_{22} & \cdots & h_{2 k} \\
% & h_{32} & \cdots & h_{3 k} \\
% & \ddots & \vdots \\
% & & & h_{k+1, k}
% \end{array}\right]
\end{split}
$$


___

**Details 2**

- Notation: $\mathrm{span} Q_k:=\mathrm{span}\{q_i\}_{i=1}^{j}$ span of columns of $Q_k$.
- $\mathrm{span} Q_k = K_{k-1}$
  - the columns of $Q_k$, $q_j$'s, span the $k$-dimensional Krylov space, $K_{k-1}$.
  <!-- - $q_{j+1}$'s are products of Gram-Schmidt of $\{q_1, q_2,\cdots, A^{j-1}r\}=\{r, Ar, A^2 r,\cdots, A^{j-1}r\}$ -->
  - E.g., in the iteration for $q_{3}$, we have 
    - $\mathrm{span}\{q_1\}=\mathrm{span}\{r\}$, and
    - $\mathrm{span}\{q_1, q_2\}=\mathrm{span}\{q_1,  Aq_1\}=\mathrm{span}\{r,  Aq_1\}=\mathrm{span}\{r,  Ar\}$, 
      - where the first equality holds since $q_{j+1} \in \mathrm{span} \{q_i\}_{i=1}^{j} \cup \{Aq_j\}$ ($j=1$), and the second and third $q_1\in \mathrm{span} \{r\}$.
    - $\mathrm{span}\{q_1, q_2, q_3\}=\mathrm{span}\{q_1, q_2, Aq_2\}=\mathrm{span}\{r, Ar, A^2r\}$.
      - where the first equality holds since $q_{j+1} \in \mathrm{span} \{q_i\}_{i=1}^{j} \cup \{Aq_j\}$ ($j=2$), and the second equality is true because $q_1\in \mathrm{span} \{r\}$ and $q_2\in \mathrm{span}\{r,  Ar\}$.


**Details 3**

For an iteration $k$, the iterate $x_k$ minimize residual over $x_0 + K_{k-1}$, 



- Observation 1: decompose $x_k = x_0 + x_{\text {add }}$ so that $x_{\text {add }}\in K_{k-1}$, 
  - then the residual can be written $r_k = b-Ax_k = b-Ax_0 -Ax_{\text {add }} = r_0 - Ax_{\text {add }}$
  - $x_{\text {add }} = Q_k c$ for some $c\in R^k$ (since $x_{\text {add }}\in K_{k-1} = \mathrm{span}Q_k$)
  - The best approximation of $x$, in the sense of minimizing residual, among $x_k \in x_0 + K_{k-1}$, equivalently among $x_{\text {add }}\in K_{k-1}$, is obtained by a least square problem.


- Observation 2: $\| r_k \|_2 =\| Q_{k+1}^T r- H_k c \|_2 $
  -  $\| r_k \|_2 = \| b - Ax_k \|_2 = \| b - Ax_0 - A x_{\text {add }} \|_2 = \| r - A x_{\text {add }} \|_2 = \| r- AQ_k c \|_2 = \| r- Q_{k+1}H_k c \|_2 = \| Q_{k+1}^T r - H_k c \|_2 $, 
  - where we used $AQ_k = Q_{k+1}H_k$ in the 3rd equality, and 
  - in the last equality, we used $r- Q_{k+1}H_k c \in\mathrm{span} Q_{k+1}$: $r\in K_0\subset K_{k}=\mathrm{span} Q_{k+1}$ and $Q_{k+1}(H_k c) \in \mathrm{span} Q_{k+1}$ (as a linear combination columns of $Q_{k+1}$) and a 
  - lemma $\| v \|_2 = \| Q^T v \|_2$ if columns of $Q$ are mutually orthonormal and $v\in\mathrm{span} Q$ (HW problem)


- Observation 3: $Q_{k+1}^T r=\left[\begin{array}{llll}\|r\|_2 & 0 & 0 & \ldots 0\end{array}\right]^T$ (length $k+1$)
  - $q_1^T r = \| r \|_2$ since $q_1=r /\|r\|_2$ and 
  - $q_j^T r = q_j^T (\|r\|_2 q_1) = 0$ for $2\le j \le k+1$: all but the first column of $Q_{k+1}$ is orthogonal to $q_1 \parallel r$.
- Combine Observation 2 and Observation 3, then
  - $x_{\text {add }}$ minimizes $\| \left[\begin{array}{llll}\|r\|_2 & 0 & 0 & \ldots 0\end{array}\right]^T - H_k c \|_2$, in other words,
  - $x_{\text {add }}$ is the least square solution of $\tilde A \tilde x = \tilde b$, where
    - $\tilde A = H_k$
    - $\tilde b = \left[\begin{array}{llll}\|r\|_2 & 0 & 0 & \ldots 0\end{array}\right]^T$ (length $k+1$).
    - $\left[\begin{array}{cccc}h_{11} & h_{12} & \cdots & h_{1 k} \\ h_{21} & h_{22} & \cdots & h_{2 k} \\ & h_{32} & \cdots & h_{3 k} \\ & & \ddots & \vdots \\ & & & h_{k+1, k}\end{array}\right]\left[\begin{array}{c}c_1 \\ c_2 \\ \\ \vdots \\ c_k\end{array}\right]=\left[\begin{array}{c}\|r\|_2 \\ 0 \\ \\ \vdots \\ 0\end{array}\right]$

**Remark** (Sauer (2017) p. 237)

- (Lest square subproblem) 
  - The least square subproblem is the most computationally expensive part of the algorithm.
    - $k$ will be small compared to the total problem size n in most applications. 
    - The least squares step should be carried out only when an approximate solution $x_k$ is needed.
      - It is independent of other part of the algorithm, and the algorithm can proceed without computing $x_{\text{add}}=Q_k$.
      - Therefore it may be done only intermittently, or at the extreme, only at the end.
- (Gram-Schmidt subroutine)
  - The Gram–Schmidt orthogonalization step carried out in the inner loop can be substituted with Householder orthogonalization at slightly increased computational complexity, if conditioning is a significant issue.
- (Exact solution)
  - In the special case $h_{k+1,k} = 0$, the least squares problem becomes square, and the approximate solution $x_k$ is exact. ()


**Remark** (Sauer (2017) p. 237)

- (Residual descent) Backward error $\|b − Ax_k \|_2$ decreases monotonically with $k$. 
  - $\|b − Ax_k \|_2 = \| r − A x_{\text{add}}\|_2$ minimized over bigger and bigger $k$-dimensional Krylov space.
- (Restart) $Q_k$ is $n \times k$ and not guaranteed to be sparse. Thus memory considerations may also limit the number k of GMRES steps.
  - In practice, one discards $Q_k$ and restart GMRES from the beginning, using the current best guess $x_k$ as the new $x_0$.
  - Restarting also helps in terms of orthogonality: $A^k r$ points to similar directions as $k$ increases.
    - Orthogonalization part of the algorithm is crucial for this reason.
- (Main application) The typical use of GMRES is for a large and sparse n × n matrix A.
  - In theory, the algorithm terminates after n steps at the correct solution x as long as A is nonsingular. 
  - In most cases, however, the goal is to run the method for k steps, where k is much smaller than n.

___

### Computation

Computation of GMRES is a topic of computational project.

## Nonlinear least square

**Theorem** (Vector dot product rule)

Let $u\left(x_1, \ldots, x_n\right)$ and $v\left(x_1, \ldots, x_n\right)$ be $\mathbb{R}^n$-vector-valued functions, and let $A\left(x_1, \ldots, x_n\right)$ be an $n \times n$ matrix function. The dot product $u^T v$ is a scalar function. Then, we have


$$
\nabla\left(u^T v\right)=v^T D u+u^T D v,
$$

and

$$
D(A v)=A \cdot D v+\sum_{i=1}^n v_i D a_i,
$$

where $a_i$ denotes the $i$ th column of $A$.

### Appendix

#### More information on GMRES

- Convergence
  - (Pessimistic) For every nonincreasing sequence $a_1, \cdots, a_{m−1}, a_m = 0$, one can find a matrix A such that the $\|r_n\| = a_n$ for all $n$, where $r_n$ is the $n$-th residual. In particular, it is possible to find a matrix for which the residual stays constant for $m − 1$ iterations, and only drops to zero at the last iteration. (Reference: [Wikipedia](https://en.wikipedia.org/wiki/Generalized_minimal_residual_method#Convergence) and also the instructor heard this in a plenary talk of a very reliable conference, though details not remembered. I remember I got surprised by the fact that even solving a linear system can be inherently difficult.)
  - (Optimistic in practice) In practice, though, GMRES often performs well. This can be proven in specific situations. 
- Relationship with MINRES
  - MINRES is similar to Conjugate Gradient (CG) method, but it assumes the matrix to be only symmetric, allowing indefinite matrices, whereas CG assumes it to be symmetric positive definite.
  - The GMRES method is essentially a generalization of MINRES for arbitrary matrices. (See the technical remark for more detials if interested.)
- $H_k$ appearing in the GMRES is an upper (nonsquare) *Hessenberg matrix*. 
  - A upper Hessenberg matrix has zero entries below tridiaginal.
  - This remark is meant to familarize with terminology.
  

###### Technical remarks

**Remark** (Technical remarks on GMRES)

- (GMRES and MINRES) The GMRES method is essentially a generalization of MINRES for arbitrary matrices. Both minimize the 2-norm of the residual and do the same calculations in exact arithmetic when the matrix is symmetric. MINRES is a short-recurrence method with a constant memory requirement, whereas GMRES requires storing the whole Krylov space, so its memory requirement is roughly proportional to the number of iterations. On the other hand, GMRES tends to suffer less from loss of orthogonality. (Reference: Wikipedia, one of whose the original references is broken; treat this remark as advice, but not as truth before confirmation.)
- (No Krylov space; further study needed) The Arnoldi iteration (computations for $q_j$'s) reduces to the Lanczos iteration for symmetric matrices. The corresponding Krylov subspace method is the minimal residual method (MinRes) of Paige and Saunders. 
    - Unlike the unsymmetric case, the MinRes method is given by a three-term recurrence relation. 
    - It can be shown that there is no Krylov subspace method for general matrices, which is given by a short recurrence relation and yet minimizes the norms of the residuals, as GMRES does. (Reference: [Wikipedia](https://en.wikipedia.org/wiki/Generalized_minimal_residual_method#Comparison_with_other_solvers))
- (Hessenberg matrix) Any matrix is unitarily similar to Henssenberg. (Reference: [Wikipedia](https://en.wikipedia.org/wiki/Hessenberg_matrix))
    - The validity of this statement is trivial by *Schur triangularization* or *Schur decomposition* ([Schur decomposition Wikipedia](https://en.wikipedia.org/wiki/Schur_decomposition)).
  - When triangularization is needed, computing a Hessenberg matrix, then moving on to a triangular matrix is more efficient. (See more detailed remark on [Wikipedia](https://en.wikipedia.org/wiki/Hessenberg_matrix#Computer_programming))
- (Convergence of GMRES) 
  - According to Greenbaum, Pták and Strakoš states that for every nonincreasing sequence $a_1, \cdots, a_{m−1}, a_m = 0$, one can find a matrix A such that the $\|r_n\| = a_n$ for all $n$, where $r_n$ is the $n$-th residual. In particular, it is possible to find a matrix for which the residual stays constant for $m − 1$ iterations, and only drops to zero at the last iteration. (Reference: [Wikipedia](https://en.wikipedia.org/wiki/Generalized_minimal_residual_method#Convergence) and also the instructor heard this in a plenary talk of a very reliable conference, though details not remembered. I remember I got surprised by the fact that even solving a linear system can be inherently difficult.)
  - In practice, though, GMRES often performs well. This can be proven in specific situations. If the symmetric part of $A$, that is $\left(A^T+A\right) / 2$, is positive definite, then
$$
\left\|r_n\right\| \leq\left(1-\frac{\lambda_{\min }^2\left(1 / 2\left(A^T+A\right)\right)}{\lambda_{\max }\left(A^T A\right)}\right)^{n / 2}\left\|r_0\right\|,
$$
where $\lambda_{\min }(M)$ and $\lambda_{\max }(M)$ denote the smallest and largest eigenvalue of the matrix $M$, respectively. 

If $A$ is symmetric and positive definite, then we even have
$$
\left\|r_n\right\| \leq\left(\frac{\kappa_2(A)^2-1}{\kappa_2(A)^2}\right)^{n / 2}\left\|r_0\right\| .
$$
where $\kappa_2(A)$ denotes the condition number of $A$ in the Euclidean norm.

In the general case, where $A$ is not positive definite, we have
$$
\frac{\left\|r_n\right\|}{\|b\|} \leq \inf _{p \in P_n}\|p(A)\| \leq \kappa_2(V) \inf _{p \in P_n} \max _{\lambda \in \sigma(A)}|p(\lambda)|,
$$
where $P_n$ denotes the set of polynomials of degree at most $n$ with $p(0)=1, V$ is the matrix appearing in the spectral decomposition of $A$, and $\sigma(A)$ is the spectrum of $A$. Roughly speaking, this says that fast convergence occurs when the eigenvalues of $A$ are clustered away from the origin and $A$ is not too far from normality. (Reference: [Wikipedia](https://en.wikipedia.org/wiki/Generalized_minimal_residual_method#Convergence), whose original reference is Lloyd N. Trefethen and David Bau, III, Numerical Linear Algebra, Society for Industrial and Applied Mathematics, 1997)

###### Deferred Details

**Remark** (Why $x_0 + K_{k-1}$, not $K_{k-1}$?)

- If we seek approximate solution $x_k$ directly from $K_{k-1}$, we have some issues in Observation 2.
  - Now, $x_k\in K_{k-1}=\mathrm{span} Q_{k}$, so $x_k=Q_k c$ for some $c\in R^k$. 

$$
\begin{split}
\| r_k \|_2 = \| b - Ax_k \|_2 = \| b- AQ_k c \|_2 = \| b- Q_{k+1}H_k c \|_2 \\
(= \| Q_{k+1}^T r - H_k c \|_2?) 
\end{split}
$$

- The last equality in question may not hold.
  - Before, we had $r$ in place $b$ and $r\in K_{k}=\mathrm{span} Q_{k+1}$ was used.
  - However, there is no guarantee that $b\in K_{k}=\mathrm{span} Q_{k+1}$.
- Also, even if this issue could be somehow resolved, this leads to more computation because Observation 3 does not hold for $b$. We had
  - $Q_{k+1}^T r=\left[\begin{array}{llll}\|r\|_2 & 0 & 0 & \ldots 0\end{array}\right]^T$ (length $k+1$), but now
  - $Q_{k+1}^T b$ must be computed.

---
This work is licensed under [Creative Commons Attribution-ShareAlike 4.0 International](https://creativecommons.org/licenses/by-sa/4.0/)